# Test Case LVV-T1600
This case will verify that the integration of the Camera Hexapod with SAL.
The blocks below represent the steps of the test case.

Requirements
* EFD
* Hexapod(s)powered on
* Thermal sensors attached to the six actuators of the hexapod.
* CSC running
* KAFKA producer running

This test will require manual verification of certain events and telemetry in the summit EFD.
Also manual verification of appropriate temperatures for each actuator.

In [1]:
from lsst.ts import salobj
from lsst.ts.idl.enums import MTHexapod
import logging
import asyncio
import os
import yaml

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd
import astropy.units as u

from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5

from lsst_efd_client import EfdClient

from lsst.ts.idl.enums.MTHexapod import ApplicationStatus

#Auxillary telescope pointing component
from lsst.ts.idl.enums import ATPtg

In [2]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])

file:///home/hdrass/WORK/ts_ddsconfig/config/ospl-sp.xml
summit


In [3]:
#This step should fail at NCSA since there is only one domain
print(os.environ["LSST_DDS_DOMAIN_ID"])

11


In [4]:
print(os.environ["LSST_DDS_INTERFACE"])

net1


The following block sets the necessary environment variables for setting up the DDS/SAL communication

In [5]:
#Wait for history data [sec] default value is 60
#os.environ["LSST_DDS_DOMAIN"] = 'citest'
#os.environ["LSST_DDS_HISTORYSYNC"] = "10"

In [6]:
STD_WAIT = 39 # This is the amount of time to wait in between a move just to make sure that the actuators are not overheating

In [7]:
STD_WAIT = 70 # This is the amount of time to wait in between a move just to make sure that the actuators are not overheating

In [8]:
STD_WAIT = 120 # This is the amount of time to wait in between moves make sure that the actuators are not overheating

In [9]:
log = logging.getLogger(__name__) # This sets up the logger for the test.
#logging.setLevel(20)
logging.basicConfig(level=logging.INFO)

In [9]:
start_time = datetime.now()
test_message = "Camera Hexapod Integration Test"
#What means index=1 here?
#script = salobj.Controller("Script", index=1)
script = salobj.Controller("Script", index=42658886)
await asyncio.sleep(10) #wait 10 second may help with DDS problems; closing all other kernels may help too
print(f'time to start is {datetime.now() - start_time} [s]')

INFO:Script:Read historical data in 0.00 sec


time to start is 0:00:11.626065 [s]


In [10]:
await script.start_task

This is how you start the remote for the CSC.

In [11]:
#Status/Error codes seen during the testing (especially during unplugging and re-plugging cables)
print("Code:4124", ApplicationStatus(4124))
print("Code:4116",ApplicationStatus(4116))
print("Code:20",ApplicationStatus(20))
print("Code:20",ApplicationStatus(512))
print("Code:5126",ApplicationStatus(5126))
print("Code:5124",ApplicationStatus(5124))
print("Code:4100",ApplicationStatus(4100))
print("Code:20484",ApplicationStatus(20484))
print("Code:21508",ApplicationStatus(21508))
print("Code:21516",ApplicationStatus(21516))
print("Code:21540",ApplicationStatus(21540))
print("Code:29701",ApplicationStatus(29701))
print("Code:28676",ApplicationStatus(28676))
print("Code:29700",ApplicationStatus(29700))

Code:4124 ApplicationStatus.DDS_CONNECTED|SYNC_MODE|RELATIVE_MOVE_MODE|EUI_CONNECTED
Code:4116 ApplicationStatus.DDS_CONNECTED|SYNC_MODE|EUI_CONNECTED
Code:20 ApplicationStatus.SYNC_MODE|EUI_CONNECTED
Code:20 ApplicationStatus.ETHERCAT_PROBLEM
Code:5126 ApplicationStatus.DDS_CONNECTED|DDS_COMMAND_SOURCE|EUI_CONNECTED|MOVE_COMPLETE
Code:5124 ApplicationStatus.DDS_CONNECTED|DDS_COMMAND_SOURCE|EUI_CONNECTED
Code:4100 ApplicationStatus.DDS_CONNECTED|EUI_CONNECTED
Code:20484 ApplicationStatus.SIMULINK_FAULT|DDS_CONNECTED|EUI_CONNECTED
Code:29701 ApplicationStatus.SIMULINK_FAULT|DRIVE_FAULT|DDS_CONNECTED|DDS_COMMAND_SOURCE|EUI_CONNECTED|FOLLOWING_ERROR
Code:28676 ApplicationStatus.SIMULINK_FAULT|DRIVE_FAULT|DDS_CONNECTED|EUI_CONNECTED
Code:29700 ApplicationStatus.SIMULINK_FAULT|DRIVE_FAULT|DDS_CONNECTED|DDS_COMMAND_SOURCE|EUI_CONNECTED


In [12]:
# create the remote
#index=1 is the camera hexapod!! index=2 is the M2 hexapod!
csc_index = 1
hexapod_csc = salobj.Remote(name="MTHexapod", domain=script.domain, index=csc_index)

INFO:MTHexapod:Read historical data in 0.16 sec
ERROR:MTHexapod.electrical:electrical DDS read queue is full (100 elements); data may be lost
ERROR:MTHexapod.application:application DDS read queue is full (100 elements); data may be lost
ERROR:MTHexapod.actuators:actuators DDS read queue is full (100 elements); data may be lost


In [13]:
await hexapod_csc.start_task

This is to verify that the CSC is running

In [14]:
await hexapod_csc.evt_heartbeat.next(flush=True, timeout=5)

General command to transition the CSC and clear errors:

In [15]:
await salobj.set_summary_state(hexapod_csc, salobj.State.OFFLINE)

[<State.STANDBY: 5>, <State.OFFLINE: 4>]

In [16]:
await salobj.set_summary_state(hexapod_csc, salobj.State.STANDBY)

[<State.OFFLINE: 4>, <State.STANDBY: 5>]

In [37]:
await salobj.set_summary_state(hexapod_csc, salobj.State.DISABLED)

RuntimeError: Error on cmd=cmd_start, initial_state=5: msg='Command failed', ackcmd=(ackcmd private_seqNum=1950132031, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Failed: final state is <State.STANDBY: 5> instead of <State.DISABLED: 1>')

In [17]:
#This is to bring the state machine in the right starting state
await salobj.set_summary_state(hexapod_csc, salobj.State.ENABLED)

[<State.DISABLED: 1>, <State.ENABLED: 2>]

The next four blocks bring the CSC to the Enabled state.

In [82]:
await hexapod_csc.cmd_enterControl.set_start(timeout=10.) # Send the enterControl command notice the casing and the set_start method

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=800082492, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Rejected: initial state is <State.ENABLED: 2> instead of <State.OFFLINE: 4>')

In [83]:
await hexapod_csc.cmd_exitControl.set_start(timeout=10.)

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=971118982, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Rejected: initial state is <State.ENABLED: 2> instead of <State.STANDBY: 5>')

In [84]:
await hexapod_csc.cmd_start.set_start(timeout=10.) 

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1929235665, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Rejected: initial state is <State.ENABLED: 2> instead of <State.STANDBY: 5>')

In [81]:
await hexapod_csc.cmd_enable.set_start(timeout=10.)

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=723732714, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Rejected: initial state is <State.ENABLED: 2> instead of <State.DISABLED: 1>')

In [33]:
await hexapod_csc.cmd_clearError.set_start(timeout=10.) # This clears the error

In [36]:
hexapod_csc.cmd_disable.set_start(timeout=10.)

<coroutine object RemoteCommand.set_start at 0x7fbc898dcd40>

In [35]:
hexapod_csc.cmd_standby.set_start(timeout=10.)

<coroutine object RemoteCommand.set_start at 0x7fbc898dc940>

In [18]:
hexapod_csc.cmd_setLogLevel.set_start(timeout=10., Level=10)

<coroutine object RemoteCommand.set_start at 0x7fbc89ec9940>

In [19]:
print(hexapod_csc)
#Bo:Check which hexapod we are controlling
whichHex1 = await hexapod_csc.evt_configuration.aget(timeout=10.)
print("Hexapod hex", whichHex1.MTHexapodID)
print(hexapod_csc.salinfo.index)

Hexapod hex 1
1


These blocks can be used to move the hexapod and check the lookup table correction:

In [20]:
#Check some Hexapod configurations 
hexConfig = await hexapod_csc.evt_configuration.aget(timeout=10.)
print("pivot at (%.0f, %.0f, %.0f) microns "%(hexConfig.pivotX, hexConfig.pivotY, hexConfig.pivotZ))
print("maxXY = ", hexConfig.maxXY, "microns, maxZ= ", hexConfig.maxZ, " microns")
print("maxUV = ", hexConfig.maxUV, "deg, maxW= ", hexConfig.maxW, " deg")

pivot at (0, 0, -2758400) microns 
maxXY =  11400.0 microns, maxZ=  13100.0  microns
maxUV =  0.36 deg, maxW=  0.1  deg


In [21]:
camHexPosition0 = await hexapod_csc.tel_actuators.aget(timeout=10.)
print(camHexPosition0)

MTHexapodID: 1, private_revCode: 50d92b35, private_sndStamp: 1619120196.6222649, private_rcvStamp: 1619120196.6228337, private_seqNum: 3218178, private_identity: MTHexapod:1, private_origin: 3230, private_host: 0, calibrated: [-442.1809999999998, -21.228999999997612, -414.4789999999954, -416.8089999999971, -18.481000000000748, -416.4749999999995], raw: [39557819.0, 39978771.0, 39585521.00000001, 39583191.0, 39981519.0, 39583525.0]


In [22]:
print("camera Hexapod positions", camHexPosition0.MTHexapodID,  camHexPosition0.calibrated)

camera Hexapod positions 1 [-442.1809999999998, -21.228999999997612, -414.4789999999954, -416.8089999999971, -18.481000000000748, -416.4749999999995]


In [23]:
#This cell makes sure that the data go to EFD
#may need to wait a few seconds before event shows up in EFD
#client = EfdClient('ncsa_teststand_efd')
client = EfdClient('summit_efd')

In [24]:
end = Time(datetime.now())
start = end - timedelta(hours=3)
dfe = await client.select_time_series('lsst.sal.MTHexapod.logevent_summaryState', '*', start.tai, end.tai, csc_index)

In [25]:
dfe

,MTHexapodID,priority,private_host,private_identity,private_kafkaStamp,private_origin,private_rcvStamp,private_revCode,private_seqNum,private_sndStamp,summaryState
2021-04-22 19:29:31.831000+00:00,1,0,0,MTHexapod:1,1.619120e+09,3230,1.619120e+09,959a0a03,10,1.619120e+09,3
2021-04-22 19:35:15.167000+00:00,1,0,0,MTHexapod:1,1.619120e+09,3230,1.619120e+09,959a0a03,11,1.619120e+09,5
2021-04-22 19:35:57.967000+00:00,1,0,0,MTHexapod:1,1.619120e+09,3230,1.619120e+09,959a0a03,12,1.619120e+09,1
2021-04-22 19:36:04.224000+00:00,1,0,0,MTHexapod:1,1.619120e+09,3230,1.619120e+09,959a0a03,13,1.619120e+09,2


In [26]:
def enabled_substate_callback(evt):
    """Print the enabled substate when event is received."""
    print(MTHexapod.EnabledSubstate(evt.enabledSubstate))

In [27]:
def in_position_callback(evt):
    """Print the in position event when it is received."""
    print(evt.in_position)

This next block will make sure that the CSC is ready for the test

In [28]:
connected = await hexapod_csc.evt_connected.aget(timeout=10.) # This is how you get an event/telemetry with await remote.type_name.aget() type being [evt,tel] and name being the name of the topic
#commandable = connected.command
telemetry_working = connected.telemetry

if not connected or not telemetry_working:
    raise Exception("Hexapod not connected or telemetry not being received.") 

TimeoutError: 

In [29]:
controller_state = await hexapod_csc.evt_controllerState.aget(timeout=10.)
state = controller_state.controllerState
offline_substate = controller_state.offlineSubstate
print(state)
#This is only for the real system. The simulater only has the AVAILABLE offline state 

'''  
if not state == salobj.State.OFFLINE and not offline_substate == MTHexapod.OfflineSubstate.AVAILABLE:
    raise Exception("Controller must be changed to Available Offline Substate.")
'''
commandable_by_dds = await hexapod_csc.evt_commandableByDDS.aget()
dds_state = commandable_by_dds.state
if not dds_state:
    raise Exception("Controller must in CommandableByDDS state.")

2


# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [26]:
# Step skiped see deviation LVV-T1802 Test Step 5                                  
#hexapod_csc.evt_controllerState.callback = enabled_substate_callback                                
#await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=7730, u=0.17,v=0,w=0,sync=True)

In [38]:
#Command used to stop the hexapod 
await hexapod_csc.cmd_stop.set_start()

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=2051466082, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Not enabled')

In [71]:
#Command used to stop the hexapod 
await hexapod_csc.cmd_abort.set_start()

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=37582138, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Not supported by this CSC')

In [31]:
#This command is to set the Hexapod to zero position
now = datetime.now()
script.log.info(f"START -- {test_message} -- LVV-T1600 -- Move to xyz 0 -- Starting Time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

INFO:Script:START -- Camera Hexapod Integration Test -- LVV-T1600 -- Move to xyz 0 -- Starting Time: 2021-04-22 19:37:10.491043 UTC


In [21]:
script.log.info(f"TEST to see if annotations like this arrive in the EFD.")

INFO:Script:TEST to see if annotations like this arrive in the EFD.


In [31]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 Test Step 7 - Starting time: {now} UTC")

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

INFO:Script:START- Camera Hexapod Integration Test -- LVV-T1600 Test Step 7 - Starting time: 2021-04-16 16:23:05.495944 UTC


hex position
    517.83     -426.62      185.01        0.01       -0.02       -0.00  



In [33]:
hexapod_csc.evt_controllerState.callback = enabled_substate_callback                                
await hexapod_csc.cmd_move.set_start(x=500,y=-500,z=200, u=0.01,v=-0.015,w=0,sync=True)
await asyncio.sleep(15.)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

script.log.info(f"STOP- {test_message} -- LVV-T1600 Test Step 7")

EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY


INFO:Script:STOP- Camera Hexapod Integration Test -- LVV-T1600 Test Step 7


hex position
    499.64     -500.78      200.06        0.01       -0.01       -0.00  



###### This command is to set the Hexapod to zero position
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 -- Move to Zero- Starting time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

In [37]:
await asyncio.sleep(STD_WAIT)
# thermal sensors

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [38]:
# thermal sensors

This block moves the hexapod to a large position but stops it after waiting 3 seconds

In [39]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 Test Step 11- Starting time: {now} UTC")

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

INFO:Script:START- Camera Hexapod Integration Test -- LVV-T1600 Test Step 11- Starting time: 2021-03-25 18:19:37.357092 UTC


hex position
      0.12        0.07        0.02       -0.00       -0.00        0.00  



In [40]:
#Deviation for broken actuator number 6: z direciton changed from 5000 to -5000
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=-5000,u=0,v=0,w=0,sync=True)

await asyncio.sleep(3)
await hexapod_csc.cmd_stop.set_start()

await asyncio.sleep(15.)

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')


#cs = await hexapod_csc.evt_controllerState.aget()
#print(MTHexapod.EnabledSubstate(cs.enabledSubstate))
#if cs.enabledSubstate != MTHexapod.EnabledSubstate.CONTROLLED_STOPPING:
#     raise Exception(f"Controller's substate is not {MTHexapod.EnabledSubstate.CONTROLLED_STOPPING}. It is {MTHexapod.EnabledSubstate(cs.enabledSubstate)}")

# cs = await hexapod_csc.evt_controllerSubstate.aget()
# print(MTHexapod.EnabledSubstate(cs.enabledSubstate))
# if cs.enabledSubstate != MTHexapod.EnabledSubstate.STATIONARY:
#     raise Exception(f"Controller's substate is not {MTHexapod.EnabledSubstate.STATIONARY}. It is {MtHexapod.EnabledSubstate(cs.enabledSubstate)}")

script.log.info(f"STOP- {test_message} -- LVV-T1600 Test Step 11")

EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY


INFO:Script:STOP- Camera Hexapod Integration Test -- LVV-T1600 Test Step 11


hex position
      0.38       -1.13    -1503.11       -0.00       -0.00        0.00  



In [41]:
#Step 15
await asyncio.sleep(STD_WAIT)

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [34]:
#This command is to set the Hexapod to zero position
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 -- Move to Zero- Starting time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

INFO:Script:START- Camera Hexapod Integration Test -- LVV-T1600 -- Move to Zero- Starting time: 2021-04-16 16:23:41.719757 UTC


EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.STATIONARY


In [ ]:
await asyncio.sleep(STD_WAIT)

CAVE: The setCompensationMode expects infrormation from the MTmount(elevation) and MTRotator(rotation).

To use the setCompensationMode at the summit the controller for MTMount needs to run and the actual MTRotator CSC needs to available at the summit.

To test this at NCSA the simulators must be stopped first.
Use the notebook MTMount-Controller the simulator, start the controller and reverse the steps after finishing testing.

For MTRotator the controller can not be used at NCSA. The simulator is always sending data even in OfflinePublishOnly state. Use the simulator at NCSA and the actual component at the summit

Start test for the lookup table compensation

In [ ]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 Test Step 17- Starting time: {now} UTC")



Use test from BO to test the LUT compensation: https://github.com/lsst-ts/ts_notebooks/blob/tickets/DM-28409/bxin/ptg2hex/hex_diagnostics.ipynb

In [ ]:
#This command is obsolete
#await hexapod_csc.cmd_moveLUT.set_start(az=180,elev=60, temp=10)


In [ ]:
'''
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=800,u=0,v=0,w=0,sync=True)
await asyncio.sleep(10)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')
'''

In [16]:
#'''
#Switch CompensationMode on
await hexapod_csc.cmd_setCompensationMode.set_start(enable=True)

lutMode = await hexapod_csc.evt_compensationMode.aget(timeout=10)
print("compsensation mode enabled?",lutMode.enabled)

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')
await asyncio.sleep(10.)
#'''

compsensation mode enabled? True
hex position
      0.21       -0.20       -0.01       -0.00       -0.00        0.00  



In [ ]:
#move the hexapod Not allowed here! First leave compensation mode than move to new postion!!! Otherwise the you hexpod gets stuck in compensation  mode
#await hexapod_csc.cmd_move.set_start(x=0,y=0,z=800,u=0,v=0,w=0,sync=True)

#await asyncio.sleep(10.)

In [ ]:
'''
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

#Switch CompensationMode off 
await hexapod_csc.cmd_setCompensationMode.set_start(enable=False)
lutMode = await hexapod_csc.evt_compensationMode.aget(timeout=10)
print("compsensation mode enabled?",lutMode.enabled)
'''

In [18]:
await hexapod_csc.cmd_setCompensationMode.set_start(enable=False)

In [ ]:
'''pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')
'''

In [ ]:
'''
posC = await hexapod_csc.evt_compensatedPosition.aget(timeout=10.)
print('Compensated position')
for i in 'xyzuvw':
    print('%10.2f '%getattr(posC, i), end= ' ')
print('\n')

posU = await hexapod_csc.evt_uncompensatedPosition.aget(timeout=10.)
print('Uncompensated position')
for i in 'xyzuvw':
    print('%10.2f '%getattr(posU, i), end= ' ')
print('\n')
'''

Analysis of the LUT correction

In [ ]:
'''LUTfile = '%s/notebooks/ts_config_mttcs/MTHexapod/v1/default.yaml'%(os.environ["HOME"])
print(LUTfile)
with open(LUTfile, 'r') as stream:
    aa = yaml.safe_load(stream)
if hexapod_csc.salinfo.index == 1:
    elevCoeff = aa['camera_config']['elevation_coeffs']
    tCoeff = aa['camera_config']['temperature_coeffs']
elif hexapod_csc.salinfo.index == 2:
    elevCoeff = aa['m2_config']['elevation_coeffs']
    tCoeff = aa['m2_config']['temperature_coeffs']
'''

In [ ]:
'''pred = []
elev=89.999
print('Predicted LUT compensation:')
for i in range(6):
    coeff = elevCoeff[i] #starts with C0
    mypoly = np.poly1d(coeff[::-1]) #poly1d needs C5 first
    pred.append(mypoly(elev))
    print('%10.2f '%pred[i], end= ' ')
print('\n')
print('Actual LUT compensation:')
for i in 'xyzuvw':
    print('%10.2f '%getattr(posC, i), end= ' ')
print('\n')
'''

In [ ]:
script.log.info(f"STOP- {test_message} -- LVV-T1600 Test Step 17")

In [ ]:
#Step 22
await asyncio.sleep(STD_WAIT)
# thermal sensors

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [43]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 Test Step 24 - Starting time: {now} UTC")

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

INFO:Script:START- Camera Hexapod Integration Test -- LVV-T1600 Test Step 24 - Starting time: 2021-03-25 18:26:29.385677 UTC


hex position
      0.23       -0.60        0.33       -0.00       -0.00        0.00  



In [44]:
await hexapod_csc.cmd_move.set_start(x=500,y=800,z=200,u=0,v=0,w=0,sync=True)
await asyncio.sleep(10.)

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
hex position
    500.01      800.37      200.19        0.00        0.00        0.00  



In [67]:
await hexapod_csc.cmd_offset.set_start(x=0,y=0,z=500,u=0,v=0,w=0,sync=True)
await asyncio.sleep(10.)

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

script.log.info(f"STOP- {test_message} -- LVV-T1600 Test Step 24")


# cs = await MThexapod_csc.evt_controllerState.aget()
# if cs.enabledSubstate != MThexapod.SetEnabledSubstateParam.MOVE_POINT_TO_POINT:
#     raise Exception(f"Controller's substate is not {MThexapod.SetEnabledSubstateParam.MOVE_POINT_TO_POINT}. It is {MThexapod.SetEnabledSubstateParam(cs.enabledSubstate)}")
# cs = await MThexapod_csc.evt_controllerSubstate.aget()
# if cs.enabledSubstate != MThexapod.SetEnabledSubstateParam.STATIONARY:
#     raise Exception(f"Controller's substate is not {MThexapod.SetEnabledSubstateParam.STATIONARY}. It is {MThexapod.SetEnabledSubstateParam(cs.enabledSubstate)}")
# in_position = await MThexapod_csc.evt_inPosition.aget(timeout=10)
# if in_position.inPosition is False:
#     raise Exception("Controller not in position.")

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1142026301, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Rejected: initial state is <State.DISABLED: 1> instead of <State.ENABLED: 2>')

In [43]:
#This command is to set the Hexapod to zero position
now = datetime.now()
script.log.info(f"START -- {test_message} -- LVV-T1600 -- Move to Zero -- Starting time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

INFO:Script:START -- Camera Hexapod Integration Test -- LVV-T1600 -- Move to Zero -- Starting time: 2021-04-22 21:19:17.623419 UTC


In [47]:
#Step 27
await asyncio.sleep(STD_WAIT)

In [63]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 Test Move - Pivot test - Starting time: {now} UTC")#Pivot test
#Test before pivot point test
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0,u=0.,v=0.1,w=0.0,sync=True)
await asyncio.sleep(10.)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

INFO:Script:START- Camera Hexapod Integration Test -- LVV-T1600 Test Movw - Pivot test - Starting time: 2021-03-25 18:41:08.212398 UTC


EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
hex position
      0.12       -0.62        0.03       -0.00        0.10       -0.00  



In [64]:
#Check some Hexapod configurations 
hexConfig = await hexapod_csc.evt_configuration.aget(timeout=10.)
print("pivot at (%.0f, %.0f, %.0f) microns "%(hexConfig.pivotX, hexConfig.pivotY, hexConfig.pivotZ))
print("maxXY = ", hexConfig.maxXY, "microns, maxZ= ", hexConfig.maxZ, " microns")
print("maxUV = ", hexConfig.maxUV, "deg, maxW= ", hexConfig.maxW, " deg")

pivot at (0, 0, -2758400) microns 
maxXY =  11400.0 microns, maxZ=  13100.0  microns
maxUV =  0.36 deg, maxW=  0.1  deg


In [82]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 Test Step 29 - Pivot test - Starting time: {now} UTC")#Pivot test
#Step 29
await hexapod_csc.cmd_move.set_start(x=2000,y=-3500,z=200,u=0.01,v=-0.05,w=0.002,sync=True)
await asyncio.sleep(10.)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

INFO:Script:START- Camera Hexapod Integration Test -- LVV-T1600 Test Step 29 - Pivot test - Starting time: 2021-03-25 18:59:39.581670 UTC


EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
hex position
   1999.57    -3500.69      200.14        0.01       -0.05        0.00  



In [83]:
#Check some Hexapod configurations 
hexConfig = await hexapod_csc.evt_configuration.aget(timeout=10.)
print("pivot at (%.0f, %.0f, %.0f) microns "%(hexConfig.pivotX, hexConfig.pivotY, hexConfig.pivotZ))
print("maxXY = ", hexConfig.maxXY, "microns, maxZ= ", hexConfig.maxZ, " microns")
print("maxUV = ", hexConfig.maxUV, "deg, maxW= ", hexConfig.maxW, " deg")

pivot at (0, 0, -2758400) microns 
maxXY =  11400.0 microns, maxZ=  13100.0  microns
maxUV =  0.36 deg, maxW=  0.1  deg


In [69]:
#Step 30 -- set a new Pivot point
script.log.info(f"START- {test_message} -- LVV-T1600 Test Step 29 - Pivot set (0,0,0)- Starting time: {now} UTC")#Pivot test
await hexapod_csc.cmd_setPivot.set_start(x=0,y=0,z=0)
await asyncio.sleep(10.)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')
#await asyncio.sleep(10.)

INFO:Script:START- Camera Hexapod Integration Test -- LVV-T1600 Test Step 29 - Pivot set (0,0,0)- Starting time: 2021-04-12 18:44:36.281308 UTC


AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1661301046, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Rejected: initial state is <State.DISABLED: 1> instead of <State.ENABLED: 2>')

Pivot point changed. Values changed in the EUI and in the EFD. Changes are shown in Chronograph.  Strud. Pos. Actual unchanged -- no movement

In [85]:
#Check some Hexapod configurations 
hexConfig = await hexapod_csc.evt_configuration.aget(timeout=10.)
print("pivot at (%.0f, %.0f, %.0f) microns "%(hexConfig.pivotX, hexConfig.pivotY, hexConfig.pivotZ))
print("maxXY = ", hexConfig.maxXY, "microns, maxZ= ", hexConfig.maxZ, " microns")
print("maxUV = ", hexConfig.maxUV, "deg, maxW= ", hexConfig.maxW, " deg")

pivot at (0, 0, -2758400) microns 
maxXY =  11400.0 microns, maxZ=  13100.0  microns
maxUV =  0.36 deg, maxW=  0.1  deg


In [86]:
#Step 31
script.log.info(f"START- {test_message} -- LVV-T1600 move to  (x=2000,y=-3500,z=200,u=0.01,v=-0.05,w=0.002,sync=True) again- Pivot test - Starting time: {now} UTC")#Pivot test
await hexapod_csc.cmd_move.set_start(x=2000,y=-3500,z=200,u=0.01,v=-0.05,w=0.002,sync=True)
await asyncio.sleep(10.)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')
await asyncio.sleep(10.)

INFO:Script:START- Camera Hexapod Integration Test -- LVV-T1600 move to  (x=2000,y=-3500,z=200,u=0.01,v=-0.05,w=0.002,sync=True) again- Pivot test - Starting time: 2021-03-25 18:59:39.581670 UTC


EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
hex position
   2000.42    -3499.64      200.17        0.01       -0.05        0.00  



Position changed in the GUI and the EFD to the set value. Strud Pos Actual values changed.

In [87]:
#Check some Hexapod configurations 
hexConfig = await hexapod_csc.evt_configuration.aget(timeout=10.)
print("pivot at (%.0f, %.0f, %.0f) microns "%(hexConfig.pivotX, hexConfig.pivotY, hexConfig.pivotZ))
print("maxXY = ", hexConfig.maxXY, "microns, maxZ= ", hexConfig.maxZ, " microns")
print("maxUV = ", hexConfig.maxUV, "deg, maxW= ", hexConfig.maxW, " deg")

pivot at (0, 0, -2758400) microns 
maxXY =  11400.0 microns, maxZ=  13100.0  microns
maxUV =  0.36 deg, maxW=  0.1  deg


In [88]:
#Set the pivot point back to the original value (z=-2758400)
#await hexapod_csc.cmd_setPivot.set_start(x=0,y=0,z=-1938000)
script.log.info(f"START- {test_message} -- LVV-T1600 Test Step 29 - Pivot set (0,0,-2758400)- Starting time: {now} UTC")#Pivot test
await hexapod_csc.cmd_setPivot.set_start(x=0,y=0,z=-2758400)
await asyncio.sleep(10.)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')
script.log.info(f"STOP- {test_message} -- LVV-T1600 Test Step 29")

INFO:Script:START- Camera Hexapod Integration Test -- LVV-T1600 Test Step 29 - Pivot set (0,0,-2758400)- Starting time: 2021-03-25 18:59:39.581670 UTC
INFO:Script:STOP- Camera Hexapod Integration Test -- LVV-T1600 Test Step 29


hex position
   4407.71    -3017.71      201.28        0.01       -0.05        0.00  



Values changed in teh EUI and in the EFD. Changes are shown in Chronograph. Strud. Pos. Actual unchanged -- no movement

In [72]:
#Set the pivot point back to the original value (0.703m in um)
'''
script.log.info(f"START- {test_message} -- LVV-T1600 Test Step 29 - Pivot set (0,0,-1938000)- Starting time: {now} UTC")#Pivot test
await hexapod_csc.cmd_setPivot.set_start(x=0,y=0,z=-1938000)
#await hexapod_csc.cmd_setPivot.set_start(x=0,y=0,z=-2758400)
await asyncio.sleep(10.)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

script.log.info(f"STOP- {test_message} -- LVV-T1600 Test Step 29")

'''

'\nscript.log.info(f"START- {test_message} -- LVV-T1600 Test Step 29 - Pivot set (0,0,-1938000)- Starting time: {now} UTC")#Pivot test\nawait hexapod_csc.cmd_setPivot.set_start(x=0,y=0,z=-1938000)\n#await hexapod_csc.cmd_setPivot.set_start(x=0,y=0,z=-2758400)\nawait asyncio.sleep(10.)\npos = await hexapod_csc.tel_application.aget(timeout=10.)\nprint(\'hex position\')\nfor i in range(6):\n    print(\'%10.2f \'%pos.position[i], end= \' \')\nprint(\'\n\')\n\nscript.log.info(f"STOP- {test_message} -- LVV-T1600 Test Step 29")\n\n'

In [89]:
#This command is to set the Hexapod to zero position
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 -- Move to Zero- Starting time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

INFO:Script:START- Camera Hexapod Integration Test -- LVV-T1600 -- Move to Zero- Starting time: 2021-03-25 19:02:30.499849 UTC


EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.STATIONARY
EnabledSubstate.STATIONARY
EnabledSubstate.STATIONARY
EnabledSubstate.STATIONARY
EnabledSubstate.STATIONARY
EnabledSubstate.STATIONARY
EnabledSubstate.STATIONARY
EnabledSubstate.STATIONARY
EnabledSubstate.STATIONARY


In [ ]:
All moved back to zero.

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [42]:
# use this for the limit movement of actuator number 6
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 Endurance Test - Starting time: {now} UTC")
for i in range(40):
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=7730,u=0,v=0.17,w=0,sync=True)
    print ('move 1 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=7730,u=0,v=0.07,w=0,sync=True)
    print ('move 2 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=7730,u=0.17,v=0,w=0,sync=True)
    print ('move 3 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=7730,u=0.07,v=0,w=0,sync=True)
    print ('move 4 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=-7730,u=0,v=0.17,w=0,sync=True)
    print ('move 5 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=-7730,u=0,v=-0.17,w=0,sync=True)
    print ('move 6 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=-7730,u=0.17,v=0,w=0,sync=True)
    print ('move 7 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=-7730,u=-0.17,v=0,w=0,sync=True)
    print ('move 8 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=7730,u=0,v=0.17,w=0,sync=True)
    print ('move 9 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=7730,u=0,v=0.1,w=0,sync=True)
    print ('move 10 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=7730,u=0.17,v=0,w=0,sync=True)
    print ('move 11 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=7730,u=0.07,v=0,w=0,sync=True)
    print ('move 12 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=-7730,u=0,v=0.17,w=0,sync=True)
    print ('move 13 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=-7730,u=0,v=-0.17,w=0,sync=True)
    print ('move 14 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=-7730,u=0.17,v=0,w=0,sync=True)
    print ('move 15 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=-7730,u=-0.17,v=0,w=0,sync=True)
    print ('move 16 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)


    #await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=7730,u=0,v=0.17,w=0,sync=True)
    #print ('move 17 completed for the nth time:', i )
    #await asyncio.sleep(STD_WAIT)
    #await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=7730,u=0,v=-0.17,w=0,sync=True)
    #print ('move 18 completed for the nth time:', i )
    #await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=7730,u=0.17,v=0,w=0,sync=True)
    print ('move 19 completed for the nth time:', i ) 
    await asyncio.sleep(STD_WAIT)
    #await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=7730,u=-0.17,v=0,w=0,sync=True)
    #print ('move 20 completed for the nth time:', i )
    #await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=-7730,u=0,v=0.17,w=0,sync=True)
    print ('move 21 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=-7730,u=0,v=-0.17,w=0,sync=True)
    print ('move 22 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=-7730,u=0.17,v=0,w=0,sync=True)
    print ('move 23 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=-7730,u=-0.17,v=0,w=0,sync=True)
    print ('move 24 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=7730,u=0,v=0.17,w=0,sync=True)
    print ('move 25 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=7730,u=0,v=-0.17,w=0,sync=True)
    print ('move 26 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=7730,u=0.17,v=0,w=0,sync=True)
    print ('move 27 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=7730,u=0.07,v=0,w=0,sync=True)
    print ('move 28 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=-7730,u=0,v=0.17,w=0,sync=True)
    print ('move 29 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=-7730,u=0,v=-0.17,w=0,sync=True)
    print ('move 30 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=-7730,u=0.17,v=0,w=0,sync=True)
    print ('move 31 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=-7730,u=-0.17,v=0,w=0,sync=True)
    print ('move 32 completed for the nth time:', i )
    await asyncio.sleep(STD_WAIT)
now = datetime.now()
script.log.info(f"STOPP- {test_message} -- LVV-T1600 Endurance Test - Stopping time: {now} UTC")

INFO:Script:START- Camera Hexapod Integration Test -- LVV-T1600 Endurance Test - Starting time: 2021-04-22 21:18:13.771636 UTC


move 1 completed for the nth time: 0


CancelledError: 

In [ ]:
#DO NOT USE until actuator number 6 is fixed!!
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 Endurance Test - Starting time: {now} UTC")
for i in range(100):
    '''
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=7730,u=0,v=0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=7730,u=0,v=-0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=7730,u=0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=7730,u=-0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=-7730,u=0,v=0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=-7730,u=0,v=-0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=-7730,u=0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=-7730,u=-0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=7730,u=0,v=0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=7730,u=0,v=-0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=7730,u=0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=7730,u=-0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=-7730,u=0,v=0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=-7730,u=0,v=-0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=-7730,u=0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=-7730,u=-0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)


    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=7730,u=0,v=0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=7730,u=0,v=-0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=7730,u=0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=7730,u=-0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=-7730,u=0,v=0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=-7730,u=0,v=-0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=-7730,u=0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=-7730,u=-0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=7730,u=0,v=0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=7730,u=0,v=-0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=7730,u=0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=7730,u=-0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=-7730,u=0,v=0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=-7730,u=0,v=-0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=-7730,u=0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=-7730,u=-0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    '''
now = datetime.now()
script.log.info(f"STOPP- {test_message} -- LVV-T1600 Endurance Test - Stopping time: {now} UTC")

In [32]:
await salobj.set_summary_state(hexapod_csc, salobj.State.STANDBY) # Transition the CSC to Standby state

RuntimeError: Error on cmd=cmd_enterControl, initial_state=4: msg='Command failed', ackcmd=(ackcmd private_seqNum=800082486, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Controller has CSC commands disabled; use the EUI to enable CSC commands')

In [ ]:
await domain.close() # Close the remote connection